In [4]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
# pd.set_option('display.max_rows', 8)
!date
!whoami

%load_ext autoreload
%autoreload 2

Thu Jul 18 16:32:21 PDT 2019
ndbs


In [5]:
import sqlns_summarizer as sqs

In [6]:
# Map countries to the correct run date/subdirectory
locations_run_dates = {
    'Bangladesh': '2019_07_02_11_55_19',
#     'Burkina_Faso': '2019_07_02_11_56_40',
#     'Ethiopia': '2019_07_02_11_58_02',
#     'India': '2019_07_02_11_58_29',
#     'Nigeria': '2019_07_02_11_58_49',
    }

# Use above dictionary to map countries to the correct path for the Vivarium output to process
#E.g. /share/costeffectiveness/results/sqlns/bangladesh/2019_06_21_00_09_53
output_dir = '/share/costeffectiveness/results/sqlns/verification_and_validation'
locactions_paths = {location: f'{output_dir}/vv_{location.lower()}/{run_date}/output.hdf'
                   for location, run_date in locations_run_dates.items()}

# Read in data from different countries
locations_outputs = {location: pd.read_hdf(path) for location, path in locactions_paths.items()}

for location, output in locations_outputs.items():
    output['location'] = location
    print(location, output.shape) # Check whether all the same size or if some data is missing

Bangladesh (500, 1230)


In [16]:
all_output = pd.concat(locations_outputs.values(), copy=False, sort=False)
all_output.head()

,,total_population_untracked,total_population_tracked,total_population,mild_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_male_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_male_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_male_in_age_group_1_to_4,mild_anemia_counts_in_2020_among_female_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_female_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_female_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_female_in_age_group_1_to_4,mild_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2021_among_male_in_age_group_1_to_4

In [17]:
all_output.filter(regex='anemia').head()

,,mild_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_male_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_male_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_male_in_age_group_1_to_4,mild_anemia_counts_in_2020_among_female_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_female_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_female_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_female_in_age_group_1_to_4,mild_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2021_among_male_in_age_group_1_to_4,severe_anemia_counts_in_2021_among_male_in_age_group_1_to_4,unexpose

## Create custom category/regex for anemia since column format is different from other risks

In [14]:
custom_search_regexes = sqs.default_column_categories_to_search_regexes()
custom_search_regexes

{'input_draw': 'input_draw',
 'random_seed': 'random_seed',
 'location': 'location',
 'intervention': 'w*\\.w*',
 'run_time': 'run_time',
 'diseases_at_end': '_prevalent_cases_at_sim_end$',
 'disease_event_count': '_event_count$',
 'population': 'population',
 'person_time': '^person_time',
 'mortality': '^death',
 'total_daly': '^years_lived_with_disability$|^years_of_life_lost$',
 'yld': '^ylds',
 'yll': '^ylls',
 'categorical_risk': '_cat\\d+_exposed'}

In [15]:
custom_search_regexes['anemia'] = 'anemia'

## Create an OutputSummarizer from the data and augmented regex dict

Then check the column categories.

In [39]:
output = sqs.SQLNSOutputSummarizer(all_output, custom_search_regexes)
output.data.head()

,,total_population_untracked,total_population_tracked,total_population,mild_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_male_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_male_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_male_in_age_group_1_to_4,mild_anemia_counts_in_2020_among_female_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_female_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_female_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_female_in_age_group_1_to_4,mild_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2021_among_male_in_age_group_1_to_4

In [40]:
output.print_column_report()

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (0 categories with no matching data columns):
 []


In [23]:
output.column_categories()

['input_draw',
 'random_seed',
 'location',
 'intervention',
 'run_time',
 'diseases_at_end',
 'disease_event_count',
 'population',
 'person_time',
 'mortality',
 'total_daly',
 'yld',
 'yll',
 'categorical_risk',
 'anemia']

In [24]:
output.columns('categorical_risk')

Index(['child_stunting_cat1_exposed_in_2020_among_male_in_age_group_early_neonatal',
       'child_stunting_cat2_exposed_in_2020_among_male_in_age_group_early_neonatal',
       'child_stunting_cat3_exposed_in_2020_among_male_in_age_group_early_neonatal',
       'child_stunting_cat4_exposed_in_2020_among_male_in_age_group_early_neonatal',
       'child_stunting_cat1_exposed_in_2020_among_female_in_age_group_early_neonatal',
       'child_stunting_cat2_exposed_in_2020_among_female_in_age_group_early_neonatal',
       'child_stunting_cat3_exposed_in_2020_among_female_in_age_group_early_neonatal',
       'child_stunting_cat4_exposed_in_2020_among_female_in_age_group_early_neonatal',
       'child_stunting_cat1_exposed_in_2020_among_male_in_age_group_late_neonatal',
       'child_stunting_cat2_exposed_in_2020_among_male_in_age_group_late_neonatal',
       ...
       'child_wasting_cat3_exposed_in_2024_among_female_in_age_group_post_neonatal',
       'child_wasting_cat4_exposed_in_2024_among

In [25]:
output.columns('yld')

Index(['ylds_due_to_protein_energy_malnutrition_in_2020_among_male_in_age_group_early_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_female_in_age_group_early_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_male_in_age_group_late_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_female_in_age_group_late_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_male_in_age_group_post_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_female_in_age_group_post_neonatal',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_male_in_age_group_1_to_4',
       'ylds_due_to_protein_energy_malnutrition_in_2020_among_female_in_age_group_1_to_4',
       'ylds_due_to_lower_respiratory_infections_in_2020_among_male_in_age_group_early_neonatal',
       'ylds_due_to_lower_respiratory_infections_in_2020_among_female_in_age_group_early_neonatal',
       ...
       'ylds_due_to_diarrhe

In [26]:
output.columns('intervention')

Index(['sqlns.program_coverage'], dtype='object')

## Aggregate over random seeds

In [41]:
output.sum_over_random_seeds()

In [44]:
output.data.head()

,location,sqlns.program_coverage,input_draw,total_population_untracked,total_population_tracked,total_population,mild_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2020_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2020_among_male_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_male_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_male_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_male_in_age_group_1_to_4,mild_anemia_counts_in_2020_among_female_in_age_group_1_to_4,moderate_anemia_counts_in_2020_among_female_in_age_group_1_to_4,severe_anemia_counts_in_2020_among_female_in_age_group_1_to_4,unexposed_anemia_counts_in_2020_among_female_in_age_group_1_to_4,mild_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_early_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_late_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_male_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,moderate_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,severe_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,unexposed_anemia_counts_in_2021_among_female_in_age_group_post_neonatal,mild_anemia_counts_in_2021_among_male_in_age_group_1_to_4,moderate_anemia_cou

In [45]:
output.print_column_report()

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (0 categories with no matching data columns):
 []


In [48]:
output.subdata['intervention'].head()

,sqlns.program_coverage
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [53]:
output.subdata['intervention'].iloc[:,0].unique()

array([0.])

In [57]:
output.subdata['total_daly'].head()

,years_lived_with_disability,years_of_life_lost
0,737.463916,149447.015343
1,1032.608061,136991.237869
2,1428.527743,148201.614338
3,1458.329621,130102.701248
4,1123.587867,145635.025426


In [58]:
output.subdata['person_time'].head()

,person_time_in_2020_among_male_in_age_group_early_neonatal,person_time_in_2020_among_female_in_age_group_early_neonatal,person_time_in_2020_among_male_in_age_group_late_neonatal,person_time_in_2020_among_female_in_age_group_late_neonatal,person_time_in_2020_among_male_in_age_group_post_neonatal,person_time_in_2020_among_female_in_age_group_post_neonatal,person_time_in_2020_among_male_in_age_group_1_to_4,person_time_in_2020_among_female_in_age_group_1_to_4,person_time_in_2021_among_male_in_age_group_early_neonatal,person_time_in_2021_among_female_in_age_group_early_neonatal,person_time_in_2021_among_male_in_age_group_late_neonatal,person_time_in_2021_among_female_in_age_group_late_neonatal,person_time_in_2021_among_male_in_age_group_post_neonatal,person_time_in_2021_among_female_in_age_group_post_neonatal,person_time_in_2021_among_male_in_age_group_1_to_4,person_time_in_2021_among_female_in_age_group_1_to_4,person_time_in_2022_among_male_in_age_group_early_neonatal,person_time_in_2022_among_female_in_age_group_early_neonatal,person_time_in_2022_among_male_in_age_group_late_neonatal,person_time_in_2022_among_female_in_age_group_late_neonatal,person_time_in_2022_among_male_in_age_group_post_neonatal,person_time_in_2022_among_female_in_age_group_post_neonatal,person_time_in_2022_among_male_in_age_group_1_to_4,person_time_in_2022_among_female_in_age_group_1_to_4,person_time_in_2023_among_male_in_age_group_early_neonatal,person_time_in_2023_among_female_in_age_group_early_neonatal,person_time_in_2023_among_male_in_age_group_late_neonatal,person_time_in_2023_among_female_in_age_group_late_neonatal,person_time_in_2023_among_male_in_age_group_post_neonatal,person_time_in_2023_among_female_in_age_group_post_neonatal,person_time_in_2023_among_male_in_age_group_1_to_4,person_time_in_2023_among_female_in_age_group_1_to_4,person_time_in_2024_among_male_in_age_group_early_neonatal,person_time_in_2024_among_female_in_age_group_early_neonatal,person_time_in_2024_among_male_in_age_group_late_neonatal,person_time_in_2024_among_female_in_age_group_late_neonatal,person_time_in_2024_among_male_in_age_group_post_neonatal,person_time_in_2024_among_female_in_age_group_post_neonatal,person_time_in_2024_among_male_in_age_group_1_to_4,person_time_in_2024_among_female_in_age_group_1_to_4,person_time_in_2025_among_male_in_age_group_early_neonatal,person_time_in_2025_among_female_in_age_group_early_neonatal,person_time_in_2025_among_male_in_age_group_late_neonatal,person_time_in_2025_among_female_in_age_group_late_neonatal,person_time_in_2025_among_male_in_age_group_post_neonatal,person_time_in_2025_among_female_in_age_group_post_neonatal,person_time_in_2025_among_male_in_age_group_1_to_4,person_time_in_2025_among_female_in_age_group_1_to_4
0,95.359128,91.101617,281.114135,271.224805,4470.187284,4411.960370,20286.178520,19707.121743,96.206960,93.946342,285.502309,279.290301,4555.002647,4369.995541,19844.248475,19358.301752,95.689524,90.333126,284.391960,269.408233,4528.502619,4401.761916,19714.226717,19091.526141,95.508502,91.729990,283.460884,271.830143,4539.049487,4332.987814,19540.949282,18908.171707,94.146926,93.228876,278.665370,277.113440,4489.677558,4405.563895,19515.891489,18864.092752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,95.969891,90.948332,284.488910,271.107307,4599.039894,4395.209323,20353.270859,19687.399073,92.478665,89.782091,273.821277,266.290339,4438.839675,4254.117426,20086.120051,19353.835450,96.352483,89.948507,285.080110,267.173577,4485.374002,4280.307130,19727.372521,18959.494960,98.933058,91.215436,293.442753,271.681078,4628.974509,4282.801611,19474.464691,18687.525482,95.217182,90.906100,283.531168,269.808981,4606.849673,4335.738338,19588.919191,18550.208172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96.076351,89.887039,285.442913,266.967157,4586.401421,4287.248263,20336.316634,19627.050487,94.047299,93.440680,278.906044,278.513296,4477.114934,4360.802267,19972.873761,19098.398518,94.436433,91.125854,278.921722,270.777584,4449.775132,4340.380707

In [59]:
output.subdata['categorical_risk'].head()

,child_stunting_cat1_exposed_in_2020_among_male_in_age_group_early_neonatal,child_stunting_cat2_exposed_in_2020_among_male_in_age_group_early_neonatal,child_stunting_cat3_exposed_in_2020_among_male_in_age_group_early_neonatal,child_stunting_cat4_exposed_in_2020_among_male_in_age_group_early_neonatal,child_stunting_cat1_exposed_in_2020_among_female_in_age_group_early_neonatal,child_stunting_cat2_exposed_in_2020_among_female_in_age_group_early_neonatal,child_stunting_cat3_exposed_in_2020_among_female_in_age_group_early_neonatal,child_stunting_cat4_exposed_in_2020_among_female_in_age_group_early_neonatal,child_stunting_cat1_exposed_in_2020_among_male_in_age_group_late_neonatal,child_stunting_cat2_exposed_in_2020_among_male_in_age_group_late_neonatal,child_stunting_cat3_exposed_in_2020_among_male_in_age_group_late_neonatal,child_stunting_cat4_exposed_in_2020_among_male_in_age_group_late_neonatal,child_stunting_cat1_exposed_in_2020_among_female_in_age_group_late_neonatal,child_stunting_cat2_exposed_in_2020_among_female_in_age_group_late_neonatal,child_stunting_cat3_exposed_in_2020_among_female_in_age_group_late_neonatal,child_stunting_cat4_exposed_in_2020_among_female_in_age_group_late_neonatal,child_stunting_cat1_exposed_in_2020_among_male_in_age_group_post_neonatal,child_stunting_cat2_exposed_in_2020_among_male_in_age_group_post_neonatal,child_stunting_cat3_exposed_in_2020_among_male_in_age_group_post_neonatal,child_stunting_cat4_exposed_in_2020_among_male_in_age_group_post_neonatal,child_stunting_cat1_exposed_in_2020_among_female_in_age_group_post_neonatal,child_stunting_cat2_exposed_in_2020_among_female_in_age_group_post_neonatal,child_stunting_cat3_exposed_in_2020_among_female_in_age_group_post_neonatal,child_stunting_cat4_exposed_in_2020_among_female_in_age_group_post_neonatal,child_stunting_cat1_exposed_in_2020_among_male_in_age_group_1_to_4,child_stunting_cat2_exposed_in_2020_among_male_in_age_group_1_to_4,child_stunting_cat3_exposed_in_2020_among_male_in_age_group_1_to_4,child_stunting_cat4_exposed_in_2020_among_male_in_age_group_1_to_4,child_stunting_cat1_exposed_in_2020_among_female_in_age_group_1_to_4,child_stunting_cat2_exposed_in_2020_among_female_in_age_group_1_to_4,child_stunting_cat3_exposed_in_2020_among_female_in_age_group_1_to_4,child_stunting_cat4_exposed_in_2020_among_female_in_age_group_1_to_4,child_stunting_cat1_exposed_in_2021_among_male_in_age_group_early_neonatal,child_stunting_cat2_exposed_in_2021_among_male_in_age_group_early_neonatal,child_stunting_cat3_exposed_in_2021_among_male_in_age_group_early_neonatal,child_stunting_cat4_exposed_in_2021_among_male_in_age_group_early_neonatal,child_stunting_cat1_exposed_in_2021_among_female_in_age_group_early_neonatal,child_stunting_cat2_exposed_in_2021_among_female_in_age_group_early_neonatal,child_stunting_cat3_exposed_in_2021_among_female_in_age_group_early_neonatal,child_stunting_cat4_exposed_in_2021_among_female_in_age_group_early_neonatal,child_stunting_cat1_exposed_in_2021_among_male_in_age_group_late_neonatal,child_stunting_cat2_exposed_in_2021_among_male_in_age_group_late_neonatal,child_stunting_cat3_exposed_in_2021_among_male_in_age_group_late_neonatal,child_stunting_cat4_exposed_in_2021_among_male_in_age_group_late_neonatal,child_stunting_cat1_exposed_in_2021_among_female_in_age_group_late_neonatal,child_stunting_cat2_exposed_in_2021_among_female_in_age_group_late_neonatal,child_stunting_cat3_exposed_in_2021_among_female_in_age_group_late_neonatal,child_stunting_cat4_exposed_in_2021_among_female_in_age_group_late_neonatal,child_stunting_cat1_exposed_in_2021_among_male_in_age_group_post_neonatal,child_stunting_cat2_exposed_in_2021_among_male_in_age_group_post_neonatal,child_stunting_cat3_exposed_in_2021_among_male_in_age_group_post_neonatal,child_stunting_cat4_exposed_in_2021_among_male_in_age_group_post_neonatal,child_stunting_cat1_exposed_in_2021_among_female_in_age_group_post_neonatal,child_stunting_cat2_exposed_in_2021_among_female_in_age_group_post_

## Test whether you can index a dataframe by a `dict_values` object of column names

Yes, you can:

In [60]:
output.columns('random_seed')

Index(['random_seed_count'], dtype='object')

In [61]:
output.columns('input_draw')

Index(['input_draw'], dtype='object')

In [64]:
output.data[['random_seed_count', 'input_draw']].head()

,random_seed_count,input_draw
0,5,3.0
1,5,8.0
2,5,21.0
3,5,29.0
4,5,35.0


In [66]:
d = {'seed': 'random_seed_count', 'draw': 'input_draw'}
output.data[d.values()].head()

,random_seed_count,input_draw
0,5,3.0
1,5,8.0
2,5,21.0
3,5,29.0
4,5,35.0


In [67]:
d.values()

dict_values(['random_seed_count', 'input_draw'])